In [12]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from keras.layers import Input, concatenate, Dropout, Reshape, Permute, Activation, ZeroPadding2D, Cropping2D, Add
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, DepthwiseConv2D
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 16

x_train = np.load('data/x_train.npz')['data'].astype(np.float32)
y_train = np.load('data/y_train.npz')['data'].astype(np.float32)
x_val = np.load('data/x_val.npz')['data'].astype(np.float32)
y_val = np.load('data/y_val.npz')['data'].astype(np.float32)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(12706, 256, 256, 3) (12706, 256, 256, 2)
(5000, 256, 256, 3) (5000, 256, 256, 2)


In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.7, 1.3]
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

train_gen = train_datagen.flow(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_gen = val_datagen.flow(
    x_val,
    y_val,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [14]:
inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# encode
conv1 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(inputs)
# conv1 = Dropout(0.2)(conv1)
conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv1)
pool1 = MaxPooling2D(pool_size=2)(conv1)
# (128, 128, 64)

shortcut_1 = pool1

conv2 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(pool1)
# conv2 = Dropout(0.2)(conv2)
conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv2)
pool2 = MaxPooling2D(pool_size=2)(conv2)
# (64, 64, 128)

shortcut_2 = pool2

conv3 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(pool2)
# conv3 = Dropout(0.2)(conv3)
conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv3)
pool3 = MaxPooling2D(pool_size=2)(conv3)
# (32, 32, 256)

# middle depthwiseconv nodes
shortcut_3 = pool3

mid = DepthwiseConv2D(3, activation='relu', padding='same', kernel_initializer='orthogonal')(pool3)
mid = Conv2D(256, 1, activation='relu', padding='same', kernel_initializer='orthogonal')(mid)

mid = DepthwiseConv2D(3, activation='relu', padding='same', kernel_initializer='orthogonal')(mid)
mid = Conv2D(256, 1, activation='relu', padding='same', kernel_initializer='orthogonal')(mid)

mid = Add()([shortcut_3, mid])

# decode
up8 = UpSampling2D(size=2)(mid)
up8 = concatenate([up8, conv3], axis=-1)
conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(up8)
# conv8 = Dropout(0.2)(conv8)
conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv8)
# (64, 64, 128)

conv8 = Add()([shortcut_2, conv8])

up9 = UpSampling2D(size=2)(conv8)
up9 = concatenate([up9, conv2], axis=-1)
conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(up9)
# conv9 = Dropout(0.2)(conv9)
conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv9)
# (128, 128, 64)

conv9 = Add()([shortcut_1, conv9])

up10 = UpSampling2D(size=2)(conv9)
up10 = concatenate([up10, conv1], axis=-1)
conv10 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(up10)
# conv10 = Dropout(0.2)(conv10)
conv10 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv10)
# (256, 256, 32)

conv11 = Conv2D(2, 1, padding='same', activation='relu',
                kernel_initializer='he_normal', kernel_regularizer=l2(0.005))(conv10)
conv11 = Reshape((IMG_HEIGHT * IMG_WIDTH, 2))(conv11)
# (256, 256, 2)

conv11 = Activation('softmax')(conv11)

outputs = Reshape((IMG_HEIGHT, IMG_WIDTH, 2))(conv11)

model = Model(inputs=inputs, outputs=outputs)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 256, 256, 32) 896         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 256, 256, 64) 18496       conv2d_38[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_10 (MaxPooling2D) (None, 128, 128, 64) 0           conv2d_39[0][0]                  
__________________________________________________________________________________________________
conv2d_40 

In [15]:
history = model.fit_generator(train_gen,
    validation_data=val_gen,
    epochs=100,
    callbacks=[
        ModelCheckpoint('models/unet_no_drop.h5', monitor='val_acc', save_best_only=True, verbose=1),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=10, verbose=1, min_lr=1e-05)

    ]
)

Epoch 1/100
795/795 [==============================] - 189s 238ms/step - loss: 0.1887 - acc: 0.9232 - val_loss: 0.1309 - val_acc: 0.9515

Epoch 00001: val_acc improved from -inf to 0.95152, saving model to models/unet_no_drop.h5
Epoch 2/100
795/795 [==============================] - 190s 240ms/step - loss: 0.1109 - acc: 0.9572 - val_loss: 0.0933 - val_acc: 0.9653

Epoch 00002: val_acc improved from 0.95152 to 0.96530, saving model to models/unet_no_drop.h5
Epoch 3/100
795/795 [==============================] - 191s 241ms/step - loss: 0.0833 - acc: 0.9688 - val_loss: 0.0690 - val_acc: 0.9749

Epoch 00003: val_acc improved from 0.96530 to 0.97489, saving model to models/unet_no_drop.h5
Epoch 4/100
795/795 [==============================] - 192s 241ms/step - loss: 0.0679 - acc: 0.9749 - val_loss: 0.0603 - val_acc: 0.9779

Epoch 00004: val_acc improved from 0.97489 to 0.97787, saving model to models/unet_no_drop.h5
Epoch 5/100
795/795 [==============================] - 192s 241ms/step - lo


Epoch 00082: val_acc did not improve from 0.98530
Epoch 83/100
795/795 [==============================] - 190s 240ms/step - loss: 0.0170 - acc: 0.9940 - val_loss: 0.0745 - val_acc: 0.9848

Epoch 00083: val_acc did not improve from 0.98530
Epoch 84/100
795/795 [==============================] - 189s 238ms/step - loss: 0.0170 - acc: 0.9940 - val_loss: 0.0747 - val_acc: 0.9848

Epoch 00084: val_acc did not improve from 0.98530
Epoch 85/100
795/795 [==============================] - 189s 238ms/step - loss: 0.0170 - acc: 0.9940 - val_loss: 0.0746 - val_acc: 0.9848

Epoch 00085: val_acc did not improve from 0.98530
Epoch 86/100
795/795 [==============================] - 191s 240ms/step - loss: 0.0170 - acc: 0.9940 - val_loss: 0.0744 - val_acc: 0.9848

Epoch 00086: val_acc did not improve from 0.98530
Epoch 87/100
795/795 [==============================] - 191s 240ms/step - loss: 0.0170 - acc: 0.9940 - val_loss: 0.0745 - val_acc: 0.9848

Epoch 00087: val_acc did not improve from 0.98530
Epoc